# Distributed data parallel MaskRCNN training with TensorFlow2 and SMDataParallel

SMDataParallel is a new capability in Amazon SageMaker to train deep learning models faster and cheaper. SMDataParallel is a distributed data parallel training framework for TensorFlow, PyTorch, and MXNet.

This notebook example shows how to use SMDataParallel with TensorFlow(version 2.4.1) on [Amazon SageMaker](https://aws.amazon.com/sagemaker/) to train a MaskRCNN model on [COCO 2017 dataset](https://cocodataset.org/#home) using [Amazon FSx for Lustre file-system](https://aws.amazon.com/fsx/lustre/) as data source.

The outline of steps is as follows:

1. Stage COCO 2017 dataset in [Amazon S3](https://aws.amazon.com/s3/)
2. Create Amazon FSx Lustre file-system and import data into the file-system from S3
3. Build Docker training image and push it to [Amazon ECR](https://aws.amazon.com/ecr/)
4. Configure data input channels for SageMaker
5. Configure hyper-prarameters
6. Define training metrics
7. Define training job, set distribution strategy to SMDataParallel and start training

**NOTE:**  With large traning dataset, we recommend using [Amazon FSx](https://aws.amazon.com/fsx/) as the input filesystem for the SageMaker training job. FSx file input to SageMaker significantly cuts down training start up time on SageMaker because it avoids downloading the training data each time you start the training job (as done with S3 input for SageMaker training job) and provides good data read throughput.


**NOTE:** This example requires SageMaker Python SDK v2.X.

## Amazon SageMaker Initialization

Initialize the notebook instance. Get the aws region, sagemaker execution role.

The IAM role arn used to give training and hosting access to your data. See the [Amazon SageMaker Roles](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html) for how to create these. Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the sagemaker.get_execution_role() with the appropriate full IAM role arn string(s). As described above, since we will be using FSx, please make sure to attach `FSx Access` permission to this IAM role.

In [ ]:
%%time
! python3 -m pip install --upgrade sagemaker
import sagemaker
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator
import boto3

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

role = get_execution_role() # provide a pre-existing role ARN as an alternative to creating a new role
print(f'SageMaker Execution Role:{role}')

client = boto3.client('sts')
account = client.get_caller_identity()['Account']
print(f'AWS account:{account}')

session = boto3.session.Session()
region = session.region_name
print(f'AWS region:{region}')

## Prepare SageMaker Training Images

1. SageMaker by default use the latest [Amazon Deep Learning Container Images (DLC)](https://github.com/aws/deep-learning-containers/blob/master/available_images.md) TensorFlow training image. In this step, we use it as a base image and install additional dependencies required for training MaskRCNN model.
2. In the Github repository https://github.com/HerringForks/DeepLearningExamples.git we have made TensorFlow-SMDataParallel MaskRCNN training script available for your use. We will be installing the same on the training image.

### Build and Push Docker Image to ECR

Run the below command build the docker image and push it to ECR.

In [ ]:
image = "<IMAGE_NAME>"  # Example: tf2-mask-rcnn-smdataparallel-sagemaker
tag = "<IMAGE_TAG>"   # Example: latest

In [ ]:
!pygmentize ./Dockerfile

In [ ]:
!pygmentize ./build_and_push.sh

In [ ]:
%%time
! chmod +x build_and_push.sh; bash build_and_push.sh {region} {image} {tag}

## Preparing FSx Input for SageMaker

1. Download and prepare your training dataset on S3.
2. Follow the steps listed here to create a FSx linked with your S3 bucket with training data - https://docs.aws.amazon.com/fsx/latest/LustreGuide/create-fs-linked-data-repo.html. Make sure to add an endpoint to your VPC allowing S3 access.
3. Follow the steps listed here to configure your SageMaker training job to use FSx https://aws.amazon.com/blogs/machine-learning/speed-up-training-on-amazon-sagemaker-using-amazon-efs-or-amazon-fsx-for-lustre-file-systems/

### Important Caveats

1. You need use the same `subnet` and `vpc` and `security group` used with FSx when launching the SageMaker notebook instance. The same configurations will be used by your SageMaker training job.
2. Make sure you set appropriate inbound/output rules in the `security group`. Specically, opening up these ports is necessary for SageMaker to access the FSx filesystem in the training job. https://docs.aws.amazon.com/fsx/latest/LustreGuide/limit-access-security-groups.html
3. Make sure `SageMaker IAM Role` used to launch this SageMaker training job has access to `AmazonFSx`.

## SageMaker TensorFlow Estimator function options

In the following code block, you can update the estimator function to use a different instance type, instance count, and distrubtion strategy. You're also passing in the training script you reviewed in the previous cell.

**Instance types**

SMDataParallel supports model training on SageMaker with the following instance types only:
1. ml.p3.16xlarge
1. ml.p3dn.24xlarge [Recommended]
1. ml.p4d.24xlarge [Recommended]

**Instance count**

To get the best performance and the most out of SMDataParallel, you should use at least 2 instances, but you can also use 1 for testing this example.

**Distribution strategy**

Note that to use DDP mode, you update the the `distribution` strategy, and set it to use `smdistributed dataparallel`.

### Training script

In the Github repository https://github.com/HerringForks/DeepLearningExamples.git we have made reference TensorFlow-SMDataParallel MaskRCNN training script available for your use. Clone the repository.

In [ ]:
# Clone herring (smdataparallel) forks repository for reference implementation of H
!rm -rf DeepLearningExamples
!git clone --recursive https://github.com/HerringForks/DeepLearningExamples.git

In [ ]:
from sagemaker.tensorflow import TensorFlow

In [ ]:
instance_type = "ml.p3dn.24xlarge" # Other supported instance type: ml.p3.16xlarge, ml.p4d.24xlarge
instance_count = 2 # You can use 2, 4, 8 etc.
docker_image = f"{account}.dkr.ecr.{region}.amazonaws.com/{image}:{tag}" # YOUR_ECR_IMAGE_BUILT_WITH_ABOVE_DOCKER_FILE
username = 'AWS'
subnets = ['<SUBNET_ID>'] # Should be same as Subnet used for FSx. Example: subnet-0f9XXXX
security_group_ids = ['<SECURITY_GROUP_ID>'] # Should be same as Security group used for FSx. sg-03ZZZZZZ
job_name = 'tf2-smdataparallel-mrcnn-fsx' # This job name is used as prefix to the sagemaker training job. Makes it easy for your look for your training job in SageMaker Training job console.
file_system_id='<FSX_ID>' # FSx file system ID with your training dataset. Example: 'fs-0bYYYYYY'


In [ ]:
SM_DATA_ROOT = '/opt/ml/input/data/train'

hyperparameters={
    "mode": "train",
    "checkpoint": '/'.join([SM_DATA_ROOT, 'model/resnet/resnet-nhwc-2018-02-07/model.ckpt-112603']), 
    "eval_samples": 5000,
    "init_learning_rate": 0.04, 
    "learning_rate_steps": "3750,5000", 
    "model_dir": "/opt/ml/code/checkpoints/tensorflow_mask_rcnn", 
    "num_steps_per_eval": 462,
    "total_steps": 500,
    "train_batch_size": 4,
    "eval_batch_size": 8,
    "training_file_pattern": '/'.join([SM_DATA_ROOT, 'train']), 
    "validation_file_pattern": '/'.join([SM_DATA_ROOT, 'val']), 
    "val_json_file": '/'.join([SM_DATA_ROOT, 'annotations/instances_val2017.json']),    
    "amp": '',
    "use_batched_nms": '',
    "xla": '',
    "nouse_custom_box_proposals_op": '',
    "seed": 987
    }

In [ ]:
estimator = TensorFlow(entry_point='DeepLearningExamples/TensorFlow2/Segmentation/MaskRCNN/mask_rcnn_sm.py',
                        role=role,
                        image_uri=docker_image,
                        source_dir='.',
                        framework_version='2.4.1',
                        py_version='py37',
                        instance_count=instance_count,
                        instance_type=instance_type,
                        sagemaker_session=sagemaker_session,
                        subnets=subnets,
                        hyperparameters=hyperparameters,
                        security_group_ids=security_group_ids,
                        debugger_hook_config=False,
                        # Training using SMDataParallel Distributed Training Framework
                        distribution={'smdistributed':{
                                            'dataparallel':{
                                                    'enabled': True
                                                 }
                                          }
                                      }
                      )

In [ ]:
# Configure FSx Input for your SageMaker Training job

from sagemaker.inputs import FileSystemInput
file_system_directory_path='YOUR_MOUNT_PATH_FOR_TRAINING_DATA' # NOTE: '/fsx/' will be the root mount path. Example: '/fsx/mask_rcnn/PyTorch'
file_system_access_mode='rw'
file_system_type='FSxLustre'
train_fs = FileSystemInput(file_system_id=file_system_id,
                                    file_system_type=file_system_type,
                                    directory_path=file_system_directory_path,
                                    file_system_access_mode=file_system_access_mode)
data_channels = {'train': train_fs}

In [ ]:
# Submit SageMaker training job
estimator.fit(inputs=data_channels, job_name=job_name)

# Additional Resources

If you are a new user of Amazon SageMaker, you may find the following helpful to understand how SageMaker uses Docker to train custom models.
* To learn more about using Amazon SageMaker with your own training image, see [Use Your Own Training Algorithms
](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-training-algo.html).

* To learn more about using Docker to train your own models with Amazon SageMaker, see [Example Notebooks: Use Your Own Algorithm or Model](https://docs.aws.amazon.com/sagemaker/latest/dg/adv-bring-own-examples.html).
* To see other examples of distributed training using Amazon SageMaker and TensorFlow, see [Distributed TensorFlow training using Amazon SageMaker
](https://github.com/awslabs/amazon-sagemaker-examples/tree/master/advanced_functionality/distributed_tensorflow_mask_rcnn).